In [9]:
import pandas as pd
import re
import requests 
from bs4 import BeautifulSoup 
import time 
import random
import lxml

In [35]:
# Read in data from github
PATH = "https://raw.githubusercontent.com/ppp-ds4a/ppp_cleaning_eda/main/data/yelp_b_webscrape.csv"
df = pd.read_csv(PATH, index_col="Unnamed: 0")

# create combined col, and rank rows based on most seen zip
df = df[df["combined"].isnull() == False]
rank_d = dict(df["postal_code"].value_counts().rank(ascending=False))
df["rank"] = df["postal_code"].map(rank_d)
df = df.sort_values("rank")

print(df.shape)
df.head(5)

(209393, 9)


,business_id,name,address,city,state,postal_code,is_open,combined,rank
36390,uaHJQ7kHmLRXh-q626flrA,Las Vegas Radiology,"3201 S Maryland Pkwy, Ste 102",Las Vegas,NV,89109,1,Las Vegas Radiology Las Vegas,1.0
111720,8C3i6-yHHriGE8abcacrgg,CSI: The Experience,3799 Las Vegas Blvd S,Las Vegas,NV,89109,1,CSI: The Experience Las Vegas,1.0
194157,Mnf10SaW1-8O3zVF_S16Dg,The Sporting House Bar & Grill,"New York New York Hotel and Casino, 3790 Las V...",Las Vegas,NV,89109,0,The Sporting House Bar & Grill Las Vegas,1.0
5124,Bh3ADEeEtMIIMe6QgrEEKw,Macy's,"The Boulevard Mall, 3634 Maryland Pkwy",Las Vegas,NV,89109,0,Macy's Las Vegas,1.0
79517,JDDg0KzpWJOO68PfMqfJbg,LoLo Rugs and Gifts,"3623 S Las Vegas Blvd, Ste 222",Las Vegas,NV,89109,1,LoLo Rugs and Gifts Las Vegas,1.0


In [30]:
# Sort rows by most seen zip code 


# df.sort_values("rank").to_csv("yelp_webscrape.csv")

### Using Regex to clean strings

- Traditional pandas methods and built-in python functions cannot clean the text appropriately
- Using multiple regular expressions can help filter text based on text patterns
- 200 status codes mean OK

##### [Regular Expression Docs](https://docs.python.org/3/library/re.html#module-re)

##### [List of HTTPS Status Codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)

In [32]:
def regex_clean(x):
    """
    Text cleaner for removing unwanted chars and replacing with appropriate chars
    """

    x = x.title()

    # Patterns
    pat1 = '[&]' # replace '&' with 'And'
    pat2 = "[' . / - ` , _]" #chars to remove

    # Regex substitution
    x = re.sub(pat1, "And", x)
    x = re.sub(pat2, "", x)

    # Split by uppercase letter then join with hyphen
    x = "-".join(re.findall('[A-Z][^A-Z]*', x))

    return "https://www.yelp.com/biz/" + x

In [36]:
# %%time
# FUNCTION TEST

# Change to adjust sample size of test
SAMPLE_SIZE = 100

# Return a list of random samples from "combined" column
sample = [x for x in df["combined"][:SAMPLE_SIZE]]

print("Sample Length: ", len(sample))

# Collect results for statistic estimation
response_dict = {}
url_404_ls = []
url_404_index_ls = []
closed_open_ls = []

# loop through samples and clean and check for reponse codes
for x in range(len(sample)):
    # Delay next execution by random integer between 1-25
    time.sleep(random.randint(9,20))

    r = requests.get(regex_clean(sample[x]))

    if str(r) == '<Response [503]>':
        print(f"503 found at iteration: {x}")
        break
    
    # If 404 then continue
    if str(r) == '<Response [404]>':
        url_404_ls.append(regex_clean(sample[x]))
        url_404_index_ls.append(df[df["combined"].str.contains(sample[x])].index[0])
        continue


    if str(r) not in response_dict:
        response_dict[str(r)] = 1
    elif str(r) in response_dict:
        response_dict[str(r)] += 1
    
    # Try to find alert saying business is closed, if not then return "no", continue if no list returned (bad request) 
    try: 
        e = "no"
        closed = BeautifulSoup(r.text, "lxml").find_all("span", {"class": "raw__373c0__3rcx7"})[0].text

        if len(closed) == 0:
            continue
    except:
        closed = e

    # Save to list whether business is open or closed
    if closed == "Yelpers report this location has closed.":
        closed_open_ls.append(0)
    else:
        closed_open_ls.append(1)

Sample Length:  100
503 found at iteration: 0


In [8]:
closed_open_ls
sample

['6600 Rue Saint-Jacques Montréal',
 '33 University Sq, Ste 270 Madison',
 '15560 N Frank Lloyd Wright Blvd Scottsdale',
 '10049 Darrow Rd Twinsburg',
 '3425 - 29th Street  NE, Suite 101 Calgary',
 '401 E Oak St Mahomet',
 '3199 S Eastern Ave Las Vegas',
 '374 College Street Toronto',
 '4701 Boulevard Lasalle Verdun',
 '10415 Centrum Pkwy Pineville',
 '2606 E Indian School Rd Phoenix',
 '28795 Lake Rd Bay Village',
 '23 College St Toronto',
 '332 Center Ave Pittsburgh',
 '160 Clairton Blvd Pittsburgh',
 '4811 N 16th St, Ste 104 Phoenix',
 '1402 Williamson St Madison',
 '10530 Northeast Pkwy Matthews',
 '27 Av Ruth Brampton',
 '6475 N Decatur Blvd, Ste 175 Las Vegas',
 '3225 W Gibson Ln Phoenix',
 '16 W Horizon Rdg Pkwy, Ste 100 Henderson',
 '15030 North Tatum Blvd, Ste 115 Phoenix',
 '6599 E Thomas Rd Scottsdale',
 '5915 S Rainbow Blvd, Ste 105 Las Vegas',
 '3628 E Indian School Rd Phoenix',
 '328 W Sahara Ave Las Vegas',
 '305 W 4th St Charlotte',
 '10752 N 89th Pl Scottsdale',
 '302 

In [7]:
df[df["combined"].str.contains(sample[x])].index[0]

191224

In [60]:
# Add URL's to orginal subbset of data used for experiment and save to CSV
df_save = df.iloc[[df[df["combined"].str.contains(sample[x])].index[0] for x in range(len(sample))]]
# df_save["url"] = url_404_ls
df_save["open"] = closed_open_ls
df_save.to_csv("url_open_close.csv")

# Look at first 20 rows
pd.read_csv("url_open_close.csv").sample(3)

ValueError: ignored

### Reponse 503 means overloaded service and the halt is temporary.  

In [55]:
print(f"Reponse Dictionary: {response_dict}\n")

# Calculate percentage of successful responses
stat_val = response_dict['<Response [200]>'] / sum(response_dict.values())
print(f"Average 'Reponse 200' over {SAMPLE_SIZE} samples: {stat_val}")

Reponse Dictionary: {'<Response [200]>': 10}

Average 'Reponse 200' over 10 samples: 1.0


### How many unique URL's

- Lower number of URL than number of original shape could be due too multiple places have the same web address like Franchises, Multiple locations, same owners, etc...

In [17]:

URL = "https://www.yelp.com/biz/skin-factory-tattoo-and-body-piercing-las-vegas" #closed
# URL = "https://www.yelp.com/biz/othello-wok-and-teriyaki-seattle" #open

r = requests.get(URL)
print(r)
BeautifulSoup(r.text, "lxml").find_all(
    "span", {"class": "raw__373c0__3rcx7"}
    )[0].text

<Response [200]>


'Yelpers report this location has closed.'

In [ ]:
# How many unique addresses
df["combined"].apply(regex_clean).nunique()

183044

### Identify Closed Banners on yelp pages

- Create functionality that looks for a closed banner determined by users and return a 0 or 1 based on whether the business is closed or open.

<p align="center">
  <img width="500" height="400" src="https://imgur.com/P1hZDTY.png">
</p>

In [1]:
!pip install lxml

In [19]:
PATH = "https://raw.githubusercontent.com/ppp-ds4a/ppp_cleaning_eda/main/data/under150k.csv"
df = pd.read_csv(PATH)
df.head(3)

,Unnamed: 0,LoanAmount,Zip,State,NAICSCode,BusinessType,JobsReported,DateApproved,Change Indicator,2017 NAICS Code,2017 NAICS Title
0,0,149990.0,17070.0,PA,721110.0,Corporation,29.0,04/13/2020,NaN,721110.0,Hotels (except Casino Hotels) and Motels
1,1,149989.0,17754.0,PA,332710.0,Corporation,0.0,08/04/2020,NaN,332710.0,Machine Shops
2,2,149987.0,19610.0,PA,541310.0,Subchapter S Corporation,11.0,04/15/2020,NaN,541310.0,Architectural Services


In [10]:
soup.find_all("span", {"class": "raw__373c0__3rcx7"})

[]

In [113]:
<div class=" alert-icon-unit__373c0__2WHUe arrange-unit__373c0__o3tjT border-color--default__373c0__3-ifU"><div class=" alert-icon-container__373c0__2Vlsq border-color--default__373c0__3-ifU"><div class=" alert-icon__373c0__5A99z border-color--default__373c0__3-ifU"><span aria-hidden="true" class="icon--24-exclamation-v2 css-p5yz4n"><svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" class="icon_svg"><path d="M12 1c6.075 0 11 4.925 11 11a11 11 0 01-11 11C5.925 23 1 18.075 1 12S5.925 1 12 1zm0 2a9 9 0 100 18 9 9 0 000-18zm0 11.39a1.5 1.5 0 110 3 1.5 1.5 0 010-3zm0-8.78a1 1 0 011 1v5.63a1 1 0 11-2 0V6.61a1 1 0 011-1z"></path></svg></span></div></div></div>

[<div id="fb-root"></div>, <div class="lang-en" id="wrap">
 <div class="page-header">
 <yelp-react-root>
 <div data-hypernova-id="ea2d8cf5-4342-4122-acfc-f54e33e204ed" data-hypernova-key="yelp_mainwww__dd7c71eacf58e5a2091de2f128716d4e89c5cc50__yelp_mainwww__ConsumerHeader__yelp_mainwww"></div>
 <script data-hypernova-id="ea2d8cf5-4342-4122-acfc-f54e33e204ed" data-hypernova-key="yelp_mainwww__dd7c71eacf58e5a2091de2f128716d4e89c5cc50__yelp_mainwww__ConsumerHeader__yelp_mainwww" type="application/json"><!--{"currentURL": "https://www.yelp.com/biz/skin-factory-tattoo-and-body-piercing-las-vegas", "userData": null, "initialHiddenLocationInputName": "find_loc", "gaConfig": {"metrics": {"www": {}, "global": {}}, "domain": "yelp.com", "user_id": null, "dimensions": {"www": {"www_second_page_pitch": [111, "status_quo"], "www_search_snippets_in_sync_with_ads": [112, "status_quo_8"], "contributions.www.remove_review_draft_modules": [183, "status_quo"], "readerx.web.biz_for_services": [116, "biz_s

In [76]:
soup.text

' \n\n\n            (function() {\n                var main = null;\n\n                var main=function(){window.onerror=function(k,a,c,i,f){var j=(document.getElementsByTagName("html")[0].getAttribute("webdriver")==="true"||navigator.userAgent==="selenium");var h=f&&(f.name==="ServerSideRenderingError"||f.name==="CSRFallbackError");if(j&&!h){document.body.innerHTML="<h1>Javascript Error Detected</h1>";var g=document.createElement("div");g.setAttribute("id","pastebinTraceback");var d=document.createElement("code");var e={message:k,source:a,lineno:c,colno:i,error:f};var b=document.createTextNode("JS ERROR DETECTED\\n"+window.JSON.stringify(e,null,2));\nd.appendChild(b);g.appendChild(d);document.getElementsByTagName("body")[0].appendChild(g)}}};\n\n                if (main === null) {\n                    throw \'invalid inline script, missing main declaration.\';\n                }\n                main();\n            })();\n    \n\n\n\n\n\n\n\n\n\n            window.yPageStart = new 

#wrap > div.main-content-wrap.main-content-wrap--full > yelp-react-root > div > div.margin-t3__373c0__1l90z.margin-b6__373c0__2Azj6.border-color--default__373c0__3-ifU > div > div > div.margin-b3__373c0__q1DuY.border-color--default__373c0__3-ifU > section:nth-child(2) > div

<div class=" alert-container__373c0__2_UFV border-color--default__373c0__3-ifU" aria-label="Error" role="alert"><div class=" alert__373c0__2W4cB page__373c0__3A_bx error__373c0__1doYa high__373c0__3qeSp arrange__373c0__2C9bH vertical-align-middle__373c0__1SDTo border-color--default__373c0__3-ifU"><div class=" alert-icon-unit__373c0__2WHUe arrange-unit__373c0__o3tjT border-color--default__373c0__3-ifU"><div class=" alert-icon-container__373c0__2Vlsq border-color--default__373c0__3-ifU"><div class=" alert-icon__373c0__5A99z border-color--default__373c0__3-ifU"><span aria-hidden="true" class="icon--24-exclamation-v2 css-p5yz4n"><svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" class="icon_svg"><path d="M12 1c6.075 0 11 4.925 11 11a11 11 0 01-11 11C5.925 23 1 18.075 1 12S5.925 1 12 1zm0 2a9 9 0 100 18 9 9 0 000-18zm0 11.39a1.5 1.5 0 110 3 1.5 1.5 0 010-3zm0-8.78a1 1 0 011 1v5.63a1 1 0 11-2 0V6.61a1 1 0 011-1z"></path></svg></span></div></div></div><div class=" arrange-unit__373c0__o3tjT arrange-unit-fill__373c0__3Sfw1 border-color--default__373c0__3-ifU"><span class=" text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa- text-size--inherit__373c0__2fB3p"><div class=" alert__373c0__2xCbt border-color--default__373c0__3-ifU"><p class=" text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa- text-size--inherit__373c0__2fB3p"><span class=" text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa- text-weight--bold__373c0__1elNz text-size--large__373c0__3t60B"><span class=" raw__373c0__3rcx7">Yelpers report this location has closed.</span></span> <span class=" text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa- text-size--large__373c0__3t60B"><span class=" raw__373c0__3rcx7">Find a <a href="/search?cflt=tattoo&amp;find_desc=&amp;find_loc=Las+Vegas%2C+NV">similar spot</a>.</span></span></p></div></span></div></div></div>

In [65]:
s = soup.find_all(
    "span", {"class": "raw__373c0__3rcx7"})

s

[]

### Notes

In [ ]:
# Read in data
yelp_business_path = "../data/yelp_dataset/yelp_academic_dataset_business.json"
yelp_b_df = pd.read_json(yelp_business_path, lines=True)

In [ ]:
# Concat name and city together
df = yelp_b_df[["name", "city"]].copy()
df["combined"] = yelp_b_df["name"] + " " + yelp_b_df["city"]

# This is the function for stripping values that we are working on
df["combined"] = df["combined"].str.strip(".-,")
df.head()

,name,city,combined
0,The Range At Lake Norman,Cornelius,The Range At Lake Norman Cornelius
1,"Carlos Santo, NMD",Scottsdale,"Carlos Santo, NMD Scottsdale"
2,Felinus,Montreal,Felinus Montreal
3,Nevada House of Hose,North Las Vegas,Nevada House of Hose North Las Vegas
4,USE MY GUY SERVICES LLC,Mesa,USE MY GUY SERVICES LLC Mesa


In [ ]:
# Save the data
# df.to_csv("web_scrape_names.csv")

In [ ]:
# https://www.yelp.com/biz/big-chickie-seattle

# The pandas apply function to append to the list
ls = []

# Emily thoughts
# chars to take care of : 
# & --> and
# + --> - / omit  
# ' --> omit
# "-" --> stays
# . --> omit / -
# , --> omit 
# / --> hyphen 
#       need to check for the presence of / in each word and replace with -
#       need to omit if it's the first char in url
# - --> stay
# array to hold characters we need to omit 
#    delete = [""]

def ls_maker(x): 
    web_string = f"https://www.yelp.com/biz/{str(x).replace(' ', '-')}"
    ls.append(web_string)
    return

df["biz_city"].apply(ls_maker)

"Hi'S"

<p class="lemon--p__373c0__3Qnnj text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa- text-size--inherit__373c0__2fB3p"><span class="lemon--span__373c0__3997G text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa- text-weight--bold__373c0__1elNz text-size--large__373c0__3t60B"><span class="lemon--span__373c0__3997G raw__373c0__3rcx7">Yelpers report this location has closed.</span></span> <span class="lemon--span__373c0__3997G text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa- text-size--large__373c0__3t60B"><span class="lemon--span__373c0__3997G raw__373c0__3rcx7">Find a <a href="/search?cflt=tattoo&amp;find_desc=&amp;find_loc=Las+Vegas%2C+NV">similar spot</a>.</span></span></p>